## Read the script

In [ ]:
import os
import numpy as np
import mne
from ieeg.io import get_data, raw_from_layout, update
from ieeg.navigate import trial_ieeg, outliers_to_nan, channel_outlier_marker, crop_empty_data
from ieeg.calc.scaling import rescale
from ieeg.viz.ensemble import chan_grid
from ieeg.timefreq.utils import crop_pad, wavelet_scaleogram
from ieeg.viz.parula import parula_map
save_dir='C:\\Users\\bl314\\Box\\CoganLab\\IndividualMeetings\\Baishen\\ieeg_results\\lexical_delay'

In [ ]:
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab")
layout = get_data("LexicalDecRepDelay", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")
subject = 'D0077'

if subject=='D0053':
    subject_Tag = 'D53'
elif subject=='D0054':
    subject_Tag = 'D54'
elif subject=='D0055':
    subject_Tag = 'D55'
elif subject=='D0057':
    subject_Tag = 'D57'
elif subject=='D0059':
    subject_Tag = 'D59'
elif subject=='D0063':
    subject_Tag = 'D63'
elif subject=='D0065':
    subject_Tag = 'D65'
elif subject=='D0066':
    subject_Tag = 'D66'
elif subject=='D0068':
    subject_Tag = 'D68'
elif subject=='D0069':
    subject_Tag = 'D69'
elif subject=='D0070':
    subject_Tag = 'D70'
elif subject=='D0071':
    subject_Tag = 'D71'
elif subject=='D0077':
    subject_Tag = 'D77'
elif subject=='D0081':
    subject_Tag = 'D81'
elif subject=='D0094':
    subject_Tag = 'D94'
elif subject=='D0096':
    subject_Tag = 'D96'
elif subject=='D0101':
    subject_Tag = 'D101'
elif subject=='D0102':
    subject_Tag = 'D102'
elif subject=='D0103':
    subject_Tag = 'D103'
elif subject=='D0107':
    subject_Tag = 'D107B'
else:
    print("Subject not found, please check.")

## Plot subj

In [ ]:
from ieeg.viz.mri import plot_subj

# plot the subject brain
fig1 = plot_subj(subject_Tag)
#Label every electrode
mne.viz.set_3d_view(fig1, azimuth=150, elevation=70, focalpoint="auto",
                    distance="auto")

## Load subj
### <span style="color:red">Please make sure that the **event.tsv** files in the **clean** derivatives do not contain **"BAD boundary"** lines, or remove them if they have</span>

In [ ]:
raw = raw_from_layout(layout.derivatives['derivatives/a'], subject=subject, desc='a',extension='.edf',preload=True)

### Remove muscle artifacts
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_spectrograms.html
### <span style="color:red">Please do the muscle artifacts removal according to the wavelet plots and previous cases in sentence repetition</span>

In [ ]:
if subject=='D0054':
    eeg_channels_to_exclude=['ROF1', 'ROF2', 'ROF3', 'ROF4', 'ROF5', 'ROF6', 'ROF7', 'ROF8', 'ROF9', 'ROF10', 'ROF11', 'ROF12', 'ROF13', 'ROF14',
 'RAMF6', 'RAMF7', 'RAMF8', 'RAMF9', 'RAMF10','RAMF11','RASF14', 'RASF15', 'RASF16','RMSF6', 'RMSF7', 'RMSF8', 'RMSF9', 'RMSF10', 'RMSF11','RPSF9', 'RPSF10', 'RPSF11', 'RPSF12', 'RPSF13', 'RPSF14','RAI5', 'RAI6', 'RAI7', 'RAI8', 'RAI9', 'RAI10', 'RAI11', 'RAI12','RPI1', 'RPI2', 'RPI3', 'RPI4', 'RPI5', 'RPI6', 'RPI7', 'RPI8', 'RPI9', 'RPI10', 'RPI11', 'RPI12', 'RPI13', 'RPI14', 'RPI15', 'RPI16','RAMT1', 'RAMT2', 'RAMT3', 'RAMT4', 'RAMT5', 'RAMT6', 'RAMT7', 'RAMT8','RMMT1', 'RMMT2', 'RMMT3', 'RMMT4', 'RMMT5', 'RMMT6', 'RMMT7', 'RMMT8', 'RMMT9', 'RMMT10', 'RMMT11', 'RMMT12','RPMT1', 'RPMT2', 'RPMT3', 'RPMT4', 'RPMT5', 'RPMT6', 'RPMT7', 'RPMT8', 'RPMT9', 'RPMT10','RPIT1', 'RPIT2', 'RPIT3', 'RPIT4', 'RPIT5', 'RPIT6', 'RPIT7', 'RPIT8', 'RPIT9', 'RPIT10', 'RPIT11', 'RPIT12']
elif subject=='D0065':
    eeg_channels_to_exclude=[]
elif subject=='D0068':
    eeg_channels_to_exclude=['LTAM1', 'LTAM2', 'LTAM3', 'LTMM1', 'LTMM2', 'LTMM3', 'LTMM4', 'LTMM5', 'LTPM1', 'LTPM2', 'LTPM3', 'LTPM4', 'LTAI1', 'LTAI2']
elif subject=='D0071':
    eeg_channels_to_exclude=['RFO14', 'RFO15', 'RFO16', 'RTAS7', 'RTAS8', 'RTAS9', 'RTAS10', 'RO16','RTAS11', 'RTAS12', 'RTMS7', 'RTMS8', 'RTMS9', 'RTMS10', 'RTMS11', 'RTMS12', 'RTPS9', 'RTPS10', 'RTAM13', 'RTAM14', 'RTAM15', 'RTAM16', 'RTMM15', 'RTMM16','RTPM15', 'RTPM16', 'RTPI15', 'RTPI16']
elif subject=='D0077':
     eeg_channels_to_exclude=['RTPM14', 'RTPM15', 'RTPI12', 'ROAI12', 'ROPI1', 'ROPI2', 'ROPI3', 'ROPI4', 'ROPI5', 'ROPI6', 'ROPI7', 'ROPI8', 'ROAM8', 'ROPM8', 'ROPM9', 'ROPM10', 'ROPM11', 'ROPM12']
elif subject=='D0094':
    eeg_channels_to_exclude=['LTP1', 'LTP2', 'LTP3', 'LTP4', 'LTP5', 'LTP6', 'LTP7', 'LTP8', 'LTP9', 'LTP10', 'LTP11', 'LTP12','LTAM5', 'LTAM6', 'LTAM7', 'LTAM8', 'LTAM9', 'LTAM10', 'LTAM11', 'LTAM12', 'LTAM13', 'LTAM14', 'LTAM15', 'LTAM16','LTAS4', 'LTAS5', 'LTAS6', 'LTAS7', 'LTAS8', 'LTAI1', 'LTAI2', 'LTAI3', 'LTAI4', 'LTAI5', 'LTAI6', 'LTAI7', 'LTAI8', 'LTAI9', 'LTAI10', 'LTAI11', 'LTAI12', 'LTMM1', 'LTMM2', 'LTMM3', 'LTMM4', 'LTMM5', 'LTMM6', 'LTMM7', 'LTMM8', 'LTMM9', 'LTMM10', 'LTMM11', 'LTMM12', 'LTMM13', 'LTMM14', 'LTPM1', 'LTPM2', 'LTPM3', 'LTPM4', 'LTPM5', 'LTPM6', 'LTPM7', 'LTPM8', 'LTPM9', 'LTPM10', 'LTPM11', 'LTPM12', 'LTPM13', 'LTPM14', 'LTPM15', 'LTPM16','LFO1', 'LFO2', 'LFO3', 'LFO4', 'LFO5', 'LFO6', 'LFO7', 'LFO8', 'LFO9', 'LFO10', 'LFO11', 'LFO12', 'LFO13', 'LFO14', 'LFO15', 'LFO16', 'LFAM12', 'LFAM13', 'LFAM14', 'LPAM14', 'LPMM16', 'LIA15', 'LIA16']
elif subject=='D0096':
    eeg_channels_to_exclude=['LTP1', 'LTP2', 'LTP3', 'LTP4', 'LTP5', 'LTP6', 'LTP7', 'LTP8', 'LTP9', 'LTP10','LTAS6', 'LTAS7', 'LTAS8',
 'LTAM1', 'LTAM2', 'LTAM3', 'LTAM4', 'LTAM5', 'LTAM6', 'LTAM7', 'LTAM8', 'LTAM9', 'LTAM10', 'LTAM11', 'LTAM12', 'LTAM13', 'LTAM14',
 'LTMM1', 'LTMM2', 'LTMM3', 'LTMM4', 'LTMM5', 'LTMM6', 'LTMM7', 'LTMM8', 'LTMM9', 'LTMM10', 'LTMM11', 'LTMM12', 'LTMM13', 'LTMM14', 'LTMM15',
 'LFO1', 'LFO2', 'LFO3', 'LFO4', 'LFO5', 'LFO6', 'LFO7', 'LFO8', 'LFO9', 'LFO10', 'LFO11', 'LFO12', 'LFO13', 'LFO14', 'LFO15','LFMM16', 'LFMM17', 'LFMM18','LFAI10']
elif subject=='D0103':
    eeg_channels_to_exclude=['LTP1', 'LTP2', 'LTP3', 'LTP4', 'LTP5', 'LTP6', 'LTP7', 'LTP8', 'LTP9', 'LTP10', 'LTP11', 'LTP12','LTAM1', 'LTAM2', 'LTAM3', 'LTAM4', 'LTAM5', 'LTAM6', 'LTAM7', 'LTAM8', 'LTAM9', 'LTAM10', 'LTAM11', 'LTAM12', 'LTAM13', 'LTAM14', 'LTAM15','LTAS7', 'LTAS8',
 'LTMM2', 'LTMM3', 'LTMM4', 'LTMM5', 'LTMM6', 'LTMM7', 'LTMM8', 'LTMM9', 'LTMM10', 'LTMM11', 'LTMM12', 'LTMM13', 'LTMM14', 'LTMM15','LTMI1', 'LTMI2', 'LTMI3', 'LTMI4', 'LTMI5', 'LTMI6', 'LTMI7', 'LTMI8', 'LTMI9', 'LTMI10', 'LTMI11', 'LTMI12', 'LTMI13', 'LTMI14','LTLI8', 'LTLI9', 'LTLI10', 'LTLI11', 'LTLI12', 'LTLI13', 'LTLI14','LFAM1', 'LFAM2', 'LFAM3', 'LFAM4', 'LFAM5', 'LFAM6', 'LFAM7', 'LFAM8', 'LFAM9', 'LFAM10', 'LFAM11', 'LFAM12', 'LFAM13', 'LFAM14', 'LFAM15','LFAI7', 'LFAI8', 'LFAI9']
elif subject=='D0107':
    eeg_channels_to_exclude=['RFOP8', 'RFOP9', 'RFOP10', 'RFOP11', 'RFOP12', 'RFOP13', 'RFOP14', 'RFOP15','A15', 'A16', 'A17', 'A18']
else:
    eeg_channels_to_exclude=[]

Sentence_rep_subj = ["D0059", "D0063",  "D0066", "D0069", "D0070", "D0081", "D0102"] # Patients with bad channels already marked in the sentence rep tasks, I chose to take them directly

if subject in Sentence_rep_subj:
    print('Sentence rep subj')
else:
    raw.info.update(bads=[b for b in eeg_channels_to_exclude if b in raw.ch_names])
    update(raw, layout, "muscle")

## Re-referenced to the average

In [ ]:
ch_type = raw.get_channel_types(only_data_chs=True)[0]
raw.set_eeg_reference(ref_channels="average", ch_type=ch_type)

## Multitaper spectrogram

In [ ]:
if not os.path.exists(os.path.join(save_dir, subject)):
    os.mkdir(os.path.join(save_dir, subject)) 
if not os.path.exists(os.path.join(save_dir, subject,'multitaper')):
    os.mkdir(os.path.join(save_dir, subject,'multitaper'))

In [ ]:
# Wavelet is good to detect and remove muscle artifact channels
# Also plot the subject brain
for task, task_Tag in zip(('Repeat','Yes_No'),('Rep','YN')):
    for epoch, t, tag in zip(
        ('Auditory_stim/'+task+'/CORRECT','Delay/'+task+'/CORRECT','Go/'+task+'/CORRECT','Resp/'+task+'/CORRECT'),
        ((-0.5, 1.5),(-0.5, 1.5),(-0.5, 1.5),(-0.5, 1)),
        ('Auditory-'+task_Tag,'Delay-'+task_Tag,'Go-'+task_Tag,'Resp-'+task_Tag)
    ):
        
        # Get the spectras
        t1 = t[0] - 0.5
        t2 = t[1] + 0.5
        times = (t1, t2)
        trials = trial_ieeg(raw, epoch, times, preload=True)
        outliers_to_nan(trials, outliers=10)
    
        
        ##############################
        ####### Multitapper ##########
        ##############################
        
        freq = np.linspace(0.5, 200, num=80)
        kwargs = dict(average=False, n_jobs=-3, freqs=freq, return_itc=False,
                      n_cycles=freq/2, time_bandwidth=4,
                      # n_fft=int(trials.info['sfreq'] * 2.75),
                      decim=20, )
                      # adaptive=True)
        spectra_multitaper = trials.compute_tfr(method="multitaper",  **kwargs)
        crop_pad(spectra_multitaper, "0.5s") # cut the first and final 0.5s, change to zero
        
                # Get the baseline
        if epoch=='Auditory_stim/'+task+'/CORRECT':
            base_multitaper = spectra_multitaper.copy().crop(-0.5, 0)
            base_multitaper = base_multitaper.average(lambda x: np.nanmean(x, axis=0), copy=True)
 
        # Baseline correction
        spectra_multitaper = spectra_multitaper.average(lambda x: np.nanmean(x, axis=0), copy=True)   
        spectra_multitaper = rescale(spectra_multitaper, base_multitaper, copy=True, mode='ratio')
        
        # Save spectras
        filename = os.path.join(save_dir, subject,'multitaper',f'{tag}-tfr.h5')
        mne.time_frequency.write_tfrs(filename, spectra_multitaper, overwrite=True)
        
        # Make spectrograms
        chan_grids=chan_grid(spectra_multitaper, size = (20,10),vlim=(0.7, 1.4), cmap=parula_map)
        
        # Save spectrograms
        fig_count=0
        for fig in chan_grids:
            figdir = os.path.join(save_dir, subject,'multitaper',f'{tag}_{fig_count+1}.jpg')
            chan_grids[fig_count].savefig(figdir,dpi=300)   
            fig_count+=1
        
        del trials,spectra_multitaper,filename
    del base_multitaper